In [1]:
import pandas as pd

In [7]:
comments = pd.read_excel('Sub-I equity eval-comments-20230907.xlsx')

In [8]:
comments['Comment'].fillna('', inplace=True)

In [2]:
subICommets = pd.read_excel('Sub-I equity eval-comments-20230907.xlsx', 'Sub-I comments')

In [3]:
subICommets['Comment'].fillna('', inplace=True)

In [7]:
studentInfo = pd.read_excel('Sub-I equity eval-20230526.xlsx')
studentInfo = studentInfo.set_index('DEid')

In [12]:
StudentIDs = comments['DeID'].unique()

In [13]:
print(len(StudentIDs))

1473


In [4]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

Embmodel = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
InfoDict = {'ANES 110': 'anes_grade', 'FCM 110': 'fcm_grade', 'MED 110': 'med_grade', 'NEURO 110': 'neuro_grade', 'OBGYN 110': 'obgyn_grade',
           'PEDS 110': 'peds_grade', 'PSYCH 110': 'psych_grade', 'SURG 110': 'surg_grade'}

In [5]:
def getUIMNNonUIMComment(commentType):
    filteredComment = subICommets[(subICommets['Comment type']==commentType)]
    merged_Comment = filteredComment.merge(studentInfo, left_on='DeID', right_index=True, how='left')
    UIM_Comment = merged_Comment[(merged_Comment['isURM']==1)]
    NonUIM_Comment = merged_Comment[(merged_Comment['isURM']==0)]
    return UIM_Comment, NonUIM_Comment

In [50]:
UIM_comment, NonUIM_comment = getUIMNNonUIMComment('Summary Comments')

In [78]:
def getUIMNNonUIMComment(courseName, commentType):
    filteredComment = subICommets[(subICommets['Comment type']==commentType) & (subICommets['Course'] ==courseName)]
    merged_Comment = filteredComment.merge(studentInfo, left_on='DeID', right_index=True, how='left')
    UIM_Comment = merged_Comment[(merged_Comment['isURM']==1)]
    NonUIM_Comment = merged_Comment[(merged_Comment['isURM']==0)]
    return UIM_Comment, NonUIM_Comment

In [79]:
UIM_SubI_comment, NonUIM_SubI_comment = getUIMNNonUIMComment('MED 140.01C', 'Constructive Comments')

In [17]:
print(UIM_comment)

        DeID core_year_x  bridges_ind_x    Course           Comment type  \
179     BAAA        2019              1  SURG 110  Constructive Comments   
180     BAAA        2019              1  SURG 110  Constructive Comments   
181     BAAA        2019              1  SURG 110  Constructive Comments   
182     BAAA        2019              1  SURG 110  Constructive Comments   
183     BAAA        2019              1  SURG 110  Constructive Comments   
...      ...         ...            ...       ...                    ...   
304694   ZZW        2019              1  SURG 110  Constructive Comments   
304695   ZZW        2019              1  SURG 110  Constructive Comments   
304889   ZZZ        2019              1  SURG 110  Constructive Comments   
304890   ZZZ        2019              1  SURG 110  Constructive Comments   
304891   ZZZ        2019              1  SURG 110  Constructive Comments   

                                                  Comment core_year_y  \
179     Gained

In [31]:
grouped_UIM_comments = UIM_comment.groupby('DeID')['Comment'].agg(list).reset_index()
grouped_NonUIM_comments = NonUIM_comment.groupby('DeID')['Comment'].agg(list).reset_index()

In [58]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=5, n_components=5, min_dist=0.005)
hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=3,
                        gen_min_span_tree=True,
                        prediction_data=True)
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords.words('english'))
from bertopic import BERTopic

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
)

In [62]:
topic_model = model.fit([x for x in UIM_comment['Comment'].values.tolist() if len(x.split(' ')) > 5])

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

2023-10-02 20:05:43,691 - BERTopic - Transformed documents to Embeddings
2023-10-02 20:05:45,846 - BERTopic - Reduced dimensionality
2023-10-02 20:05:45,985 - BERTopic - Clustered reduced embeddings


In [63]:
UIM_topic_model = topic_model
UIM_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,69,-1_xx_patients_care_patient,"[xx, patients, care, patient, skills]",[XX functioned at the level of an intern putti...
1,0,28,0_inpatient_family medicine_family_advanced in...,"[inpatient, family medicine, family, advanced ...",[On her Family Medicine Advanced Inpatient Cle...
2,1,24,1_fred_patients_care_team,"[fred, patients, care, team, firo]",[XX was outstanding in his care of patients on...
3,2,20,2_level_intern_functioned level_xx functioned,"[level, intern, functioned level, xx functione...",[XX functioned at the level of an intern durin...
4,3,20,3_xx_patient_patients_care,"[xx, patient, patients, care, plans]",[XX took on management of very medically and s...
5,4,18,4_student_student doctor_doctor xx_doctor,"[student, student doctor, doctor xx, doctor, m...",[Student doctor XX was a welcome and valued ad...
6,5,17,5_xx displayed_jon_patients_outstanding,"[xx displayed, jon, patients, outstanding, man...",[XX displayed excellent fund of knowledge/clin...
7,6,16,6_medicine sub_skills medicine_skills_xx exhib...,"[medicine sub, skills medicine, skills, xx exh...",[XX exhibited outstanding knowledge and reason...
8,7,16,7_patients_francisco_san_san francisco,"[patients, francisco, san, san francisco, gene...",[XX mastered the art of staying structured or...
9,8,16,8_joe_abdi_patients_multiple,"[joe, abdi, patients, multiple, patient]",[Joe did a great job on the medicine acting in...


In [64]:
UIM_topic_model.visualize_barchart()

In [54]:
NonUIM_comment = [x for x in NonUIM_comment['Comment'].values.tolist() if isinstance(x, str) and len(x.split(' ')) > 5]

In [55]:
NonUIM_topic_model = model.fit(NonUIM_comment)

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

2023-10-02 20:03:59,719 - BERTopic - Transformed documents to Embeddings
2023-10-02 20:04:02,720 - BERTopic - Reduced dimensionality
2023-10-02 20:04:03,179 - BERTopic - Clustered reduced embeddings


In [56]:
NonUIM_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,231,-1_xx_patients_patient_team,"[xx, patients, patient, team, care]",[XX was an unparalleled advocate fantastic com...
1,0,77,0_skills_xx exhibited_professional skills_medi...,"[skills, xx exhibited, professional skills, me...",[XX exhibited excellent knowledge and reasonin...
2,1,64,1_family medicine_family_inpatient_inpatient c...,"[family medicine, family, inpatient, inpatient...",[On her Family Medicine Advanced Inpatient Cle...
3,2,55,2_outstanding_skills_skills medicine_professio...,"[outstanding, skills, skills medicine, profess...",[XX exhibited outstanding knowledge and reason...
4,3,54,3_francisco general_san_san francisco_francisco,"[francisco general, san, san francisco, franci...",[By the end of XX's subinternship in internal ...
...,...,...,...,...,...
86,85,6,85_james_yield_notes notes_fully reliable,"[james, yield, notes notes, fully reliable, re...",[James did an excellent job on his medicine ac...
87,86,6,86_diagnostic_diagnosis_differential diagnosis...,"[diagnostic, diagnosis, differential diagnosis...",[XX functioned at the level of an intern durin...
88,87,6,87_gabe_josh_patient admitted_team structure,"[gabe, josh, patient admitted, team structure,...",[Gabe did outstanding work on his Family Medic...
89,88,6,88_alex_month alex_alex excellent_alex demonst...,"[alex, month alex, alex excellent, alex demons...",[XX (Alex) did well in the patient care domain...


In [57]:
NonUIM_topic_model.visualize_barchart()